In [2]:
import numpy as np
import pandas as pd

train = pd.read_json("./subtrain_df.json")
valid = pd.read_json("./valid_df.json")
test = pd.read_json("./test_df.json")

In [3]:
train.head()

,hotel_name,room_type,烤箱,戶外傢俱,高樓層（有電梯）,床邊插座,微波爐,防過敏,額外的廁所,兒童書籍、DVD 或音樂,...,hotel_type_經濟型飯店,hotel_type_膠囊旅館,hotel_type_青年旅館,hotel_type_飯店,hotel_type_飯店式公寓,traffic_spot,atraction_spot,traffic_min_dist,atraction_min_dist,hotel_id
0,191旅店- 寧夏分館,標準雙人房,0,0,1,1,1,0,1,0,...,0,0,0,0,0,捷運中山站,寧夏夜市,0.7,0.1,127
1,191旅店- 寧夏分館,標準雙人房－無窗,0,0,1,1,1,0,1,0,...,0,0,0,0,0,捷運中山站,寧夏夜市,0.7,0.1,127
2,191旅店- 寧夏分館,經濟雙人房－無窗,0,0,1,1,1,0,1,0,...,0,0,0,0,0,捷運中山站,寧夏夜市,0.7,0.1,127
3,191旅店- 寧夏分館,商務雙人或雙床房,0,0,1,1,1,0,1,0,...,0,0,0,0,0,捷運中山站,寧夏夜市,0.7,0.1,127
4,191旅店- 寧夏分館,商務雙人或雙床房－無窗,0,0,1,1,1,0,1,0,...,0,0,0,0,0,捷運中山站,寧夏夜市,0.7,0.1,127


* 丟掉不會拿來進行預測的Features

In [4]:
train = train.drop(['hotel_name','room_type','hotel_id','traffic_spot','atraction_spot'], axis=1)
valid = valid.drop(['hotel_name','room_type','hotel_id','traffic_spot','atraction_spot'], axis=1)
test = test.drop(['hotel_name','room_type','hotel_id','traffic_spot','atraction_spot'], axis=1)

In [5]:
train.head()

,烤箱,戶外傢俱,高樓層（有電梯）,床邊插座,微波爐,防過敏,額外的廁所,兒童書籍、DVD 或音樂,廁所,浴缸,...,hotel_type_旅館,hotel_type_民宿,hotel_type_汽車旅館,hotel_type_經濟型飯店,hotel_type_膠囊旅館,hotel_type_青年旅館,hotel_type_飯店,hotel_type_飯店式公寓,traffic_min_dist,atraction_min_dist
0,0,0,1,1,1,0,1,0,1,0,...,1,0,0,0,0,0,0,0,0.7,0.1
1,0,0,1,1,1,0,1,0,1,0,...,1,0,0,0,0,0,0,0,0.7,0.1
2,0,0,1,1,1,0,1,0,1,0,...,1,0,0,0,0,0,0,0,0.7,0.1
3,0,0,1,1,1,0,1,0,1,0,...,1,0,0,0,0,0,0,0,0.7,0.1
4,0,0,1,1,1,0,1,0,1,0,...,1,0,0,0,0,0,0,0,0.7,0.1


* 將train, valid, test data轉為np格式供訓練

In [6]:
x_train = np.array(train.drop(['y_norm','y_orig'], axis=1))
y_train_o = np.array(train['y_orig'])
y_train_n = np.array(train['y_norm'])

x_valid = np.array(valid.drop(['y_norm','y_orig'], axis=1))
y_valid_o = np.array(valid['y_orig'])
y_valid_n = np.array(valid['y_norm'])

x_test = np.array(test.drop(['y_norm','y_orig'], axis=1))
y_test_o = np.array(test['y_orig'])
y_test_n = np.array(test['y_norm'])

In [8]:
print(x_train.shape)
print(y_train_o.shape)
print(y_train_n.shape)
print(x_valid.shape)
print(y_valid_o.shape)
print(y_valid_n.shape)
print(x_test.shape)
print(y_test_o.shape)
print(y_test_n.shape)

(1884, 142)
(1884,)
(1884,)
(408, 142)
(408,)
(408,)
(353, 142)
(353,)
(353,)


* trainAll為train+valid set

In [9]:
x_trainAll = np.concatenate((x_train, x_valid))
y_trainAll_o = np.concatenate((y_train_o, y_valid_o))
y_trainAll_n = np.concatenate((y_train_n, y_valid_n))

## Regression Baseline
* 每一算法都先tune lambda/ nest，再進行一次訓練

#### Ordinary Least Squares

In [10]:
from sklearn import linear_model
reg = linear_model.LinearRegression().fit(x_trainAll, y_trainAll_o)
y_pred = reg.predict(x_test)

In [11]:
difference = (np.array(y_pred)-y_test_o)**2
RMSE = np.sqrt(np.mean(difference))
print('y_orig Test RMSE:',RMSE)

absCoef = abs(reg.coef_)
maxCoef = absCoef.argsort()[-10:]

for m in maxCoef:
    print(x.columns[m], reg.coef_[m])

y_orig Test RMSE: 64485411900.396164
bed_4 個上下舖床位 605785446100.7738
bed_1張床位 605785446100.8031
hotel_type_家庭旅館 605785446100.8995
bed_3 張日式床舖 605785446100.9185
bed_2 張日式床舖 605785446101.0309
hotel_type_經濟型飯店 605785446101.1456
hotel_type_度假屋 605785446101.1458
hotel_type_民宿 605785446101.1965
bed_6 個上下舖床位 605785446101.8812
bed_6 張日式床舖 605785446102.5873


In [48]:
reg = linear_model.LinearRegression().fit(x_trainAll, y_trainAll_n)
y_pred = reg.predict(x_test)

In [49]:
difference = (np.array(y_pred)-y_test_n)**2
RMSE = np.sqrt(np.mean(difference))
print('y_norm Test RMSE:',RMSE)

absCoef = abs(reg.coef_)
maxCoef = absCoef.argsort()[-10:]

for m in maxCoef:
    print(x.columns[m], reg.coef_[m])

y_norm Test RMSE: 50713581846.53957
hotel_type_民宿 476410848544.4341
hotel_type_旅館 476410848544.45166
bed_3 張加大雙人床 476410848544.47504
bed_2 張日式床舖 476410848544.5203
bed_3 張日式床舖 476410848544.6087
hotel_type_家庭旅館 476410848544.67456
hotel_type_飯店式公寓 476410848544.70123
hotel_type_B&B 476410848544.9535
bed_6 個上下舖床位 476410848545.33746
bed_6 張日式床舖 476410848545.7952


#### Ridge regression

In [52]:
lambda_list = [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1, 2]

besta = lambda_list[0]
best_RMSE = float("inf")
for l in lambda_list:
    # Fit and test
    reg = linear_model.Ridge(alpha=l).fit(x_train, y_train_o)
    y_pred = reg.predict(x_valid)
    # Count RMSE
    difference = (np.array(y_pred)-y_valid_o)**2
    RMSE = np.sqrt(np.mean(difference))
    
    if RMSE < best_RMSE:
        besta = l
        best_RMSE = RMSE
print("The best alpha is:", besta, "which has y_orig valid RMSE =", best_RMSE)

The best alpha is: 2 which has y_orig valid RMSE = 1.4223673681547462


In [53]:
reg = linear_model.Ridge(alpha=besta).fit(x_trainAll, y_trainAll_o)
y_pred = reg.predict(x_test)
difference = (np.array(y_pred)-y_test_o)**2
RMSE = np.sqrt(np.mean(difference))
print('y_orig Test RMSE:',RMSE)

absCoef = abs(reg.coef_)
maxCoef = absCoef.argsort()[-10:]

for m in maxCoef:
    print(x.columns[m], reg.coef_[m])

y_orig Test RMSE: 1.3134419698394422
bed_6 個上下舖床位 0.4499927019146852
筆記型電腦 0.45725475827759876
電熱毯 -0.5073336937258567
hotel_type_民宿 0.5780636581802316
完全獨立 0.6191836173683094
bed_6 張日式床舖 0.6952866778815955
hotel_type_公寓 -0.7155801995415734
bed_4 張雙人床 -1.1099291967311513
bed_3 張雙人床 -1.1877140984799286
bed_6 張單人床 -1.6998355149904276


In [26]:
besta = lambda_list[0]
best_RMSE = float("inf")
for l in lambda_list:
    # Fit and test
    reg = linear_model.Ridge(alpha=l).fit(x_train, y_train_n)
    y_pred = reg.predict(x_valid)
    # Count RMSE
    difference = (np.array(y_pred)-y_valid_n)**2
    RMSE = np.sqrt(np.mean(difference))
    
    if RMSE < best_RMSE:
        besta = l
        best_RMSE = RMSE
print("The best alpha is:", besta, "which has y_norm valid RMSE =", best_RMSE)

The best alpha is: 2 which has y_norm valid RMSE = 1.0252142483234268


In [27]:
reg = linear_model.Ridge(alpha=besta).fit(x_trainAll, y_trainAll_n)
y_pred = reg.predict(x_test)
difference = (np.array(y_pred)-y_test_n)**2
RMSE = np.sqrt(np.mean(difference))
print('y_norm Test RMSE:',RMSE)

absCoef = abs(reg.coef_)
maxCoef = absCoef.argsort()[-10:]

for m in maxCoef:
    print(x.columns[m], reg.coef_[m])

y_norm Test RMSE: 1.1978947980159234
完全獨立 0.3640310753911904
hotel_type_飯店式公寓 0.3673540719843122
bed_6 個上下舖床位 0.36945013754053524
hotel_type_膠囊旅館 -0.459535696731111
兒童書籍、DVD 或音樂 -0.49056567798963885
電動遊戲 -0.4921681339891018
bed_6 張日式床舖 0.522509687464184
bed_4 張雙人床 -0.7226513290087895
bed_3 張雙人床 -0.9322394483728533
bed_6 張單人床 -1.4102405373384326


#### Lasso Regression

In [65]:
besta = lambda_list[0]
best_RMSE = float("inf")
for l in lambda_list:
    # Fit and test
    reg = linear_model.Lasso(alpha=l).fit(x_train, y_train_o)
    y_pred = reg.predict(x_valid)
    # Count RMSE
    difference = (np.array(y_pred)-y_valid_o)**2
    RMSE = np.sqrt(np.mean(difference))
    
    if RMSE < best_RMSE:
        besta = l
        best_RMSE = RMSE
print("The best alpha is:", besta, "which has y_orig valid RMSE =", best_RMSE)

The best alpha is: 0.1 which has y_orig valid RMSE = 0.9729296673164382


In [66]:
reg = linear_model.Lasso(alpha=besta).fit(x_trainAll, y_trainAll_o)
y_pred = reg.predict(x_test)
difference = (np.array(y_pred)-y_test_o)**2
RMSE = np.sqrt(np.mean(difference))
print('y_orig Test RMSE:',RMSE)

absCoef = abs(reg.coef_)
maxCoef = absCoef.argsort()[-10:]

for m in maxCoef:
    print(x.columns[m], reg.coef_[m])

y_orig Test RMSE: 0.9852690524678075
建築物中的獨立公寓 0.0
書桌 0.0
摺疊床 0.0
休息區 0.0
地毯 0.0
免費盥洗用品 -0.0
沙發 0.0
設有連通室 0.0
燙衣設備 0.0
atraction_min_dist -0.0


* Lasso表現好但是是在猜平均

In [63]:
### y_orig
reg.intercept_

8.350105303585426

In [67]:
np.mean(y_trainAll_o)

8.350105303585426

In [57]:
besta = lambda_list[0]
best_RMSE = float("inf")
for l in lambda_list:
    # Fit and test
    reg = linear_model.Lasso(alpha=l).fit(x_train, y_train_n)
    y_pred = reg.predict(x_valid)
    # Count RMSE
    difference = (np.array(y_pred)-y_valid_n)**2
    RMSE = np.sqrt(np.mean(difference))
    
    if RMSE < best_RMSE:
        besta = l
        best_RMSE = RMSE
print("The best alpha is:", besta, "which has y_norm valid RMSE =", best_RMSE)

The best alpha is: 0.05 which has y_norm valid RMSE = 0.8544247646537696


In [58]:
reg = linear_model.Lasso(alpha=besta).fit(x_trainAll, y_trainAll_n)
y_pred = reg.predict(x_test)
difference = (np.array(y_pred)-y_test_n)**2
RMSE = np.sqrt(np.mean(difference))
print('y_norm Test RMSE:',RMSE)

absCoef = abs(reg.coef_)
maxCoef = absCoef.argsort()[-10:]

for m in maxCoef:
    print(x.columns[m], reg.coef_[m])

y_norm Test RMSE: 0.8524682315928703
書桌 0.0
燙衣設備 0.0
摺疊床 -0.0
休息區 -0.0
地毯 -0.0
免費盥洗用品 -0.0
沙發 -0.0
設有連通室 0.0
私人入口 0.0
traffic_min_dist 0.012731666726125425


* Lasso y_norm 也幾乎在猜平均

In [59]:
reg.intercept_

-0.016283161855369717

In [60]:
np.mean(y_train_n)

0.011335416698301485

#### Random Forest Regressor

In [34]:
from sklearn.ensemble import RandomForestRegressor

t_grid = []
d = np.power(1000/5, 1/9)
for i in range(10):
    t_grid.append(5*np.power(d,i)) # 5~1000, 10 points
t_grid = list(map(int, t_grid)) # number of tree must be integer
t_grid[-1] = 1000

In [35]:
best_nest = t_grid[0]
best_RMSE = float("inf")
for t in t_grid:
    regr = RandomForestRegressor(n_estimators=t, random_state=0).fit(x_train, y_train_o)
    y_pred = regr.predict(x_valid)
    # Count RMSE
    difference = (np.array(y_pred)-y_valid_o)**2
    RMSE = np.sqrt(np.mean(difference))
    
    if RMSE < best_RMSE:
        besta = l
        best_RMSE = RMSE
print("The best nest is:", best_nest, "which has y_orig valid RMSE =", best_RMSE)

The best nest is: 5 which has y_orig valid RMSE = 1.1042082889031268


In [36]:
reg = RandomForestRegressor(n_estimators=best_nest, random_state=0).fit(x_trainAll, y_trainAll_o)
y_pred = reg.predict(x_test)
difference = (np.array(y_pred)-y_test_o)**2
RMSE = np.sqrt(np.mean(difference))
print('y_orig Test RMSE:',RMSE)

absCoef = abs(reg.feature_importances_)
maxCoef = absCoef.argsort()[-10:]

for m in maxCoef:
    print(x.columns[m], reg.feature_importances_[m])

y_orig Test RMSE: 1.2195369030249958
暖氣 0.016540597951656737
bed_2 張加大雙人床 0.016826964546281877
bed_3 張雙人床 0.017189101355627033
bed_4 張雙人床 0.01783307375379899
bed_1 張雙人床 0.018400882815819784
bed_2 張單人床 0.02686141303416347
bed_2 張雙人床 0.039101388375049256
traffic_min_dist 0.050861809089151055
bed_1 張單人床 0.05587343610277318
atraction_min_dist 0.07753272376861761


In [37]:
best_nest = t_grid[0]
best_RMSE = float("inf")
for t in t_grid:
    regr = RandomForestRegressor(n_estimators=t, random_state=0).fit(x_train, y_train_n)
    y_pred = regr.predict(x_valid)
    # Count RMSE
    difference = (np.array(y_pred)-y_valid_n)**2
    RMSE = np.sqrt(np.mean(difference))
    
    if RMSE < best_RMSE:
        besta = l
        best_RMSE = RMSE
print("The best nest is:", best_nest, "which has y_norm valid RMSE =", best_RMSE)

The best nest is: 5 which has y_norm valid RMSE = 0.8605356601835166


In [39]:
reg = RandomForestRegressor(n_estimators=best_nest, random_state=0).fit(x_trainAll, y_trainAll_n)
y_pred = reg.predict(x_test)
difference = (np.array(y_pred)-y_test_n)**2
RMSE = np.sqrt(np.mean(difference))
print('y_norm Test RMSE:',RMSE)

absCoef = abs(reg.feature_importances_)
maxCoef = absCoef.argsort()[-10:]

for m in maxCoef:
    print(x.columns[m], reg.feature_importances_[m])

y_norm Test RMSE: 1.1699233741834623
景觀 0.013660282241453079
bed_2 張加大雙人床 0.0163350998264643
市景 0.01634205959109026
沙發 0.018262554350095266
淋浴間 0.025945496104376752
bed_2 張單人床 0.034867322143810345
bed_2 張雙人床 0.045080761014398577
atraction_min_dist 0.06042336220699561
bed_1 張單人床 0.06180874387997691
traffic_min_dist 0.07244052820122501


#### Gradient Boosting Regressor

In [40]:
from sklearn.ensemble import GradientBoostingRegressor

best_nest = t_grid[0]
best_RMSE = float("inf")
for t in t_grid:
    reg = GradientBoostingRegressor(n_estimators=t).fit(x_train, y_train_o)
    y_pred = reg.predict(x_valid)
    # Count RMSE
    difference = (np.array(y_pred)-y_valid_o)**2
    RMSE = np.sqrt(np.mean(difference))
    
    if RMSE < best_RMSE:
        besta = l
        best_RMSE = RMSE
print("The best nest is:", best_nest, "which has y_orig valid RMSE =", best_RMSE)

The best nest is: 5 which has y_orig valid RMSE = 0.9774975941898942


In [42]:
reg = GradientBoostingRegressor(n_estimators=best_nest).fit(x_trainAll, y_trainAll_o)
y_pred = reg.predict(x_test)
difference = (np.array(y_pred)-y_test_o)**2
RMSE = np.sqrt(np.mean(difference))
print('y_orig Test RMSE:',RMSE)

absCoef = abs(reg.feature_importances_)
maxCoef = absCoef.argsort()[-10:]

for m in maxCoef:
    print(x.columns[m], reg.feature_importances_[m])

y_orig Test RMSE: 0.9968012494089667
免費盥洗用品 0.04074560639181923
SPA 浴缸 0.04747389889609607
沖洗座 0.05271097596340403
浴袍 0.06838499430465722
加長床舖（超過兩公尺） 0.08697993613997394
保險箱 0.09276705283513513
床邊插座 0.09356082557448406
bed_6 張單人床 0.09482344989841984
暖氣 0.11341135774923702
bed_4 張雙人床 0.17288728613920584


In [43]:
best_nest = t_grid[0]
best_RMSE = float("inf")
for t in t_grid:
    reg = GradientBoostingRegressor(n_estimators=t).fit(x_train, y_train_n)
    y_pred = reg.predict(x_valid)
    # Count RMSE
    difference = (np.array(y_pred)-y_valid_n)**2
    RMSE = np.sqrt(np.mean(difference))
    
    if RMSE < best_RMSE:
        besta = l
        best_RMSE = RMSE
print("The best nest is:", best_nest, "which has y_norm valid RMSE =", best_RMSE)

The best nest is: 5 which has y_norm valid RMSE = 0.8574390878349061


In [44]:
reg = GradientBoostingRegressor(n_estimators=best_nest).fit(x_trainAll, y_trainAll_n)
y_pred = reg.predict(x_test)
difference = (np.array(y_pred)-y_test_n)**2
RMSE = np.sqrt(np.mean(difference))
print('y_norm valid Test RMSE:',RMSE)

absCoef = abs(reg.feature_importances_)
maxCoef = absCoef.argsort()[-10:]

for m in maxCoef:
    print(x.columns[m], reg.feature_importances_[m])

y_norm valid Test RMSE: 1.1507761944573431
traffic_min_dist 0.022720977319014363
按次計費頻道 0.02489700467129529
bed_2 張加大雙人床 0.02971051091883234
手部消毒液 0.04820939268056184
bed_4 張雙人床 0.04918329121670951
atraction_min_dist 0.0708464932325348
bed_3 張雙人床 0.0843806337623416
電熱水壺 0.1798832124876483
茶／咖啡沖泡設備 0.20522317085644107
bed_6 張單人床 0.26255778946785513
